# Fiddle with the EAWAG scans
Look at the orientation and see if we can do some cropping based on landmarks.

In [ ]:
import platform
import os
import glob
import pandas
import imageio
import numpy
import scipy
import k3d
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import seaborn
import dask
import dask_image.imread
from dask.distributed import Client, LocalCluster
import skimage
from tqdm.auto import tqdm, trange
import math

In [ ]:
# Set dask temporary folder
# Do this before creating a client: https://stackoverflow.com/a/62804525/323100
import tempfile
if 'Linux' in platform.system():
    tmp = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
elif 'Darwin' in platform.system():
    tmp = tempfile.gettempdir()
else:
    if 'anaklin' in platform.node():
        tmp = os.path.join('F:\\')
    else:
        tmp = os.path.join('D:\\')
dask.config.set({'temporary_directory': os.path.join(tmp, 'tmp')})
print('Dask temporary files go to %s' % dask.config.get('temporary_directory'))

In [ ]:
# Start cluster and client now, after setting tempdir
cluster = LocalCluster()
client = Client(cluster)

In [ ]:
print('You can seee what DASK is doing at "http://localhost:%s/status"' % client.scheduler_info()['services']['dashboard'])

In [ ]:
# # Ignore warnings in the notebook
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
# Set up figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
plt.rcParams['figure.figsize'] = (8, 4.5)  # Size up figures a bit
plt.rcParams['figure.dpi'] = 200

In [ ]:
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [ ]:
# Display all plots identically
lines = 3
# And then do something like
# plt.subplot(lines, int(numpy.ceil(len(Data) / float(lines))), c + 1)

In [ ]:
# Different locations if running either on Linux or Windows
FastSSD = True
overthere = False  # Load the data directly from the iee-research_storage drive
nanoct = True  # Load the data directly from the 2214
# to speed things up significantly
if 'Linux' in platform.system():
    if FastSSD:
        BasePath = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
    elif overthere:
        BasePath = os.path.join(os.sep, 'home', 'habi', 'research-storage-iee')
    elif nanoct:
        BasePath = os.path.join(os.path.sep, 'home', 'habi', '2214')
    else:
        BasePath = os.path.join(os.sep, 'home', 'habi', '1272')
elif 'Darwin' in platform.system():
    FastSSD = False
    BasePath = os.path.join('/Users/habi/Dev/EAWAG/Data')
elif 'Windows' in platform.system():
    if FastSSD:
        BasePath = os.path.join('F:\\')
    else:
        if 'anaklin' in platform.node():
            BasePath = os.path.join('S:\\')
        else:
            BasePath = os.path.join('D:\\Results')
if not overthere:
    Root = os.path.join(BasePath, 'EAWAG')
else:
    Root = BasePath
# if overthere:
#         Root = os.path.join('I:\\microCTupload')
print('We are loading all the data from %s' % Root)

In [ ]:
def get_pixelsize(logfile):
    """Get the pixel size from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Image Pixel' in line and 'Scaled' not in line:
                pixelsize = float(line.split('=')[1])
    return(pixelsize)

In [ ]:
def get_git_hash():
    '''
    Get the current git hash from the repository.
    Based on http://stackoverflow.com/a/949391/323100 and
    http://stackoverflow.com/a/18283905/323100
    '''
    from subprocess import Popen, PIPE
    import os
    gitprocess = Popen(['git',
                        '--git-dir',
                        os.path.join(os.getcwd(), '.git'),
                        'rev-parse',
                        '--short',
                        '--verify',
                        'HEAD'],
                       stdout=PIPE)
    (output, _) = gitprocess.communicate()
    return output.strip().decode("utf-8")

In [ ]:
# # Make directory for output
# OutPutDir = os.path.join(os.getcwd(), 'Output', get_git_hash())
# print('We are saving all the output to %s' % OutPutDir)
# os.makedirs(OutPutDir, exist_ok=True)

In [ ]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [ ]:
# Get *all* log files
# Sort them by time, not name
Data['LogFile'] = [f for f in sorted(glob.glob(os.path.join(Root, '**', '*.log'), recursive=True), key=os.path.getmtime)]

In [ ]:
# Generate folder name
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]

In [ ]:
# Get rid of all non-rec logfiles
for c, row in Data.iterrows():
    if 'rec' not in row.Folder:
        Data.drop([c], inplace=True)
    elif 'SubScan' in row.Folder:
        Data.drop([c], inplace=True)
    elif 'rectmp.log' in row.LogFile:
        Data.drop([c], inplace=True)
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [ ]:
# Generate us some meaningful colums
Data['Fish'] = [l[len(Root) + 1:].split(os.sep)[0] for l in Data['LogFile']]
Data['Scan'] = ['_'.join(l[len(Root) + 1:].split(os.sep)[1:-1]) for l in Data['LogFile']]

In [ ]:
Data.tail()

In [ ]:
# Get the file names of the reconstructions
Data['Reconstructions'] = [sorted(glob.glob(os.path.join(f, '*rec0*.png'))) for f in Data['Folder']]
Data['Number of reconstructions'] = [len(r) for r in Data.Reconstructions]

In [ ]:
# Drop samples which have either not been reconstructed yet or of which we deleted the reconstructions with
# `find . -name "*rec*.png" -type f -mtime +333 -delete`
# Based on https://stackoverflow.com/a/13851602
# for c, row in Data.iterrows():
#     if not row['Number of reconstructions']:
#         print('%s contains no PNG files, we might be currently reconstructing it' % row.Folder)
Data = Data[Data['Number of reconstructions'] > 0]
Data.reset_index(drop=True, inplace=True)
print('We have %s folders with reconstructions' % (len(Data)))

In [ ]:
# Get parameters we need from the log files
Data['Voxelsize'] = [get_pixelsize(log) for log in Data['LogFile']]

In [ ]:
Data.tail()

In [ ]:
Data['PreviewImagePath'] = [sorted(glob.glob(os.path.join(f, '*_spr.bmp')))[0] for f in Data['Folder']]
# Squeeze the preview images, so we don't have to do it afterwards
# https://github.com/dask/dask-image/issues/239#issuecomment-874411053
Data['PreviewImage'] = [dask_image.imread.imread(pip).squeeze()
                        if pip
                        else numpy.random.random((100, 100)) for pip in Data['PreviewImagePath']]

In [ ]:
# Set how many lines we want the images to have
lines = 4

In [ ]:
for c, row in Data.iterrows():
    plt.subplot(lines, int(numpy.ceil(len(Data) / float(lines))), c + 1)
    plt.imshow(row.PreviewImage)
    plt.title(os.path.join(row['Fish'], row['Scan']))
    plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                  'um',
                                  color='black',
                                  frameon=True))
    plt.axis('off')
plt.tight_layout()
plt.savefig(os.path.join(Root, 'ScanOverviews.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
# Load all reconstructions into DASK arrays
Reconstructions = [None] * len(Data)
for c, row in tqdm(Data.iterrows(),
                   desc='Load reconstructions',
                   total=len(Data)):
    Reconstructions[c] = dask_image.imread.imread(os.path.join(row['Folder'],
                                                               '*rec*.png'))

In [ ]:
# Check if something went wrong
# for file in Data['OutputNameRec']:
#     print(file)
#     dask.array.from_zarr(file)

In [ ]:
# How big are the datasets?
Data['Size'] = [rec.shape for rec in Reconstructions]

In [ ]:
# The three cardinal directions
# Names adapted to fishes: https://en.wikipedia.org/wiki/Fish_anatomy#Body
directions = ['Anteroposterior',
              'Lateral',
              'Dorsoventral']

In [ ]:
# Read or calculate the directional MIPs, put them into the dataframe and save them to disk
for d, direction in enumerate(directions):
    Data['MIP_' + direction] = ''
for c, row in tqdm(Data.iterrows(), desc='Calculating MIPs', total=len(Data)):
    for d, direction in tqdm(enumerate(directions),
                             desc='%s/%s: %s' % (row['Fish'], row['Scan'], direction),
                             leave=False,
                             total=len(directions)):
        outfilepath = os.path.join(os.path.dirname(row['Folder']),
                                   '%s.%s.MIP.%s.png' % (row['Fish'], row['Scan'], direction))
        if os.path.exists(outfilepath):
            Data.at[c, 'MIP_' + direction] = dask_image.imread.imread(outfilepath).squeeze()
        else:
            # Generate MIP
            Data.at[c, 'MIP_' + direction] = Reconstructions[c].max(axis=d).compute().squeeze()
            # Save it out
            imageio.imwrite(outfilepath, Data.at[c, 'MIP_' + direction].astype('uint8'))

In [ ]:
# Show MIP slices
for c, row in tqdm(Data.iterrows(),
                   desc='Saving overview of MIP images',
                   total=len(Data)):
    outfilepath = os.path.join(os.path.dirname(row['Folder']),
                               '%s.%s.MIPs.png' % (row['Fish'], row['Scan']))
    print('%s/%s: %s' % (c, len(Data), os.path.join(row.Fish, row.Scan)))
    for d, direction in tqdm(enumerate(directions),
                             desc='%s/%s' % (row['Fish'], row['Scan']),
                             total=len(directions)):
        plt.subplot(1, 3, d + 1)
        plt.imshow(row['MIP_' + direction])
        plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                      'um'))
        plt.title('%s MIP' % direction)
        plt.axis('off')
        plt.title('%s\n%s MIP' % (os.path.join(row['Fish'], row['Scan']), direction))
    if not os.path.exists(outfilepath):
        plt.savefig(outfilepath,
                    transparent=True,
                    bbox_inches='tight')
    plt.show()

In [ ]:
# for c, row in Data.iterrows():
#     # load one Anteroposterior MIP into an image for fiddling with it
#     plt.imshow(row.MIP_Anteroposterior)
#     plt.title('%s/%s: %s/%s' % (c, len(Data), row.Fish, row.Scan))
#     plt.show()

The functions below were copied from Hearts-Melly/SubMyocardAnalysis.ipynb, in which we also look at the orientation of things

In [ ]:
# From Hearts-Melly/SubMyocardAnalysis.ipynb
def get_properties(roi, verbose=False):
    # Label filled image
    labeled_img = skimage.measure.label(roi)
    # Extract regionprops of image and put data into pandas
    # https://stackoverflow.com/a/66632023/323100
    props = skimage.measure.regionprops_table(labeled_img,
                                              properties=('label',
                                                          'centroid',
                                                          'area',
                                                          'perimeter',
                                                          'orientation'))
    table = pandas.DataFrame(props)
    table_sorted = table.sort_values(by='area', ascending=False)
    # return only the region with the biggest area
    properties = table_sorted.iloc[:1].reset_index()
    if verbose:
        plt.imshow(roi, alpha=0.5)
        plt.title('Original')
        plt.axis('off')
        plt.imshow(numpy.ma.masked_equal(labeled_img, 0), cmap='viridis', alpha=0.5)
        plt.title('Labelled')
        plt.axis('off')
        plt.show()
    return(properties)

In [ ]:
def get_largest_region(segmentation, verbose=False):
    # Get out biggest item from https://stackoverflow.com/a/55110923/323100
    labels = skimage.measure.label(segmentation)
    assert(labels.max() != 0)  # assume at least 1 CC
    largestCC = labels == numpy.argmax(numpy.bincount(labels.flat)[1:]) + 1
    if verbose:
        plt.subplot(121)
        plt.imshow(segmentation)
        plt.subplot(122)
        plt.imshow(largestCC)
        plt.suptitle('Largest connected component')
        plt.show()
    return largestCC

In [ ]:
def threshold(image, verbose=False):
    # Calculate threshold of image where image is non-zero
    threshold = skimage.filters.threshold_otsu(image[image > 0])
    if verbose:
        plt.subplot(121)
        plt.imshow(image)
        plt.imshow(dask.array.ma.masked_equal(image > threshold, 0),
                   alpha=0.618,
                   cmap='viridis_r')
        plt.subplot(122)
        plt.semilogy(histogram(image), label='Log-Histogram')
        plt.axvline(threshold, label='Otsu threshold: %s' % threshold)
        plt.legend()
        plt.show()
    return(threshold)

In [ ]:
# Calculate the histogram of an image
# We can safely assume to only use 8bit images
def histogram(img):
    histogram, bins = dask.array.histogram(dask.array.array(img),
                                           bins=2**8,
                                           range=[0, 2**8])
    return(histogram)

In [ ]:
whichone = 18
print(os.path.join(Data.Fish[whichone], Data.Scan[whichone]))
img = Data.MIP_Anteroposterior[whichone]

In [ ]:
t = threshold(img.compute(), verbose=True)

In [ ]:
a = get_largest_region(img > t, verbose=True)

In [ ]:
def get_centroid(img, verbose=False):
    props = get_properties(img)
    # Drawing from https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_regionprops.html
    y0, x0 = props['centroid-0'], props['centroid-1']
    if verbose:
        plt.imshow(img)
        plt.scatter(props['centroid-1'], props['centroid-0'], marker=None, color='r')
        plt.axis('off')
        plt.show()
    return((x0, y0))

In [ ]:
def get_contour(filled_img, verbose=False):
    # Contouring from https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_regionprops.html
    largest_region = get_largest_region(filled_img, verbose=False)
    contour = skimage.measure.find_contours(largest_region)
    # Even though we look only at the largest region, we still might get out more than one contour
    # Let's thus sort the list and just continue with the longest one
    (contour).sort(key=len)
    cy, cx = contour[-1].T
    if verbose:
        plt.imshow(filled_img)
        plt.plot(cx, cy, lw=1, c='r')
        plt.axis('off')
        plt.show()
    return(cx, cy)

In [ ]:
contour = get_contour(img > t, verbose=True)

In [ ]:
centroid = get_centroid(img > t, verbose=True)

In [ ]:
def draw_orientation(img, x0, x1, x2, y0, y1, y2, self=False):
    if self:
        plt.imshow(img)
    plt.plot((x0, x1), (y0, y1), '-r', linewidth=1)
    plt.plot((x0, x2), (y0, y2), '-r', linewidth=1)
    if self:
        plt.axis('off')
        plt.show()
    return()

In [ ]:
def get_orientation(img, voxelsize, length=5000, verbose=False):
    props = get_properties(img)
    whichlengthdowewant = length
    reallength = whichlengthdowewant / voxelsize  # um
    # Drawing from https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_regionprops.htm
    x0, y0 = get_centroid(img)
    x1 = x0 + math.cos(props['orientation']) * reallength
    y1 = y0 - math.sin(props['orientation']) * reallength
    x2 = x0 - math.sin(props['orientation']) * reallength
    y2 = y0 - math.cos(props['orientation']) * reallength
    if verbose:
        plt.imshow(img)
        plt.scatter(props['centroid-1'], props['centroid-0'], marker=None, color='r')
        draw_orientation(img, x0, x1, x2, y0, y1, y2)
        plt.gca().add_artist(ScaleBar(voxelsize, 'um'))
        plt.title('Image with %s um long orientation bars' % length)
        plt.axis('off')
        plt.show()
    return(x0, x1, x2, y0, y1, y2)

In [ ]:
x0, x1, x2, y0, y1, y2 = get_orientation(img > t,
                                         Data.Voxelsize[4],
                                         length=10000,
                                         verbose=True)

The functions above were copied from Hearts-Melly/SubMyocardAnalysis.ipynb, in which we also look at the orientation of things

In [ ]:
a = get_properties(img > t)
print(a)

In [ ]:
# Use the angle and centroid to rotate image
img_rotated = numpy.empty_like(img)
img_rotated = skimage.transform.rotate(img.compute(),
                                       angle=numpy.rad2deg(a.orientation[0]),
                                       center=(a['centroid-0'][0], a['centroid-1'][0]),
                                       preserve_range=True)

In [ ]:
# Let's show what we did
plt.subplot(121)
plt.imshow(img)
plt.scatter(a['centroid-0'], a['centroid-1'], s=50)
plt.title('Original image with centroid')
plt.subplot(122)
plt.imshow(img_rotated)
plt.scatter(a['centroid-0'], a['centroid-1'], s=50)
plt.title('Image rotated by %0.f°, with centroid' % numpy.rad2deg(a.orientation[0]))
plt.show()

In [ ]:
# Detect otoliths

In [ ]:
# Copy of otolith detection function from Zebrafish-Carolina/Muscles-Volumentry
def otolither(img, sigma=5, level=0.9, threshold=None, verbose=False):
    '''
    Function to detect the otoliths in the Anteroposterior MIPs.
    We simply look for peaks in the gray values :)
    '''
    # Smooth image for less noise
    smoothed = scipy.ndimage.gaussian_filter(img, sigma=sigma, order=0)
    # If we don't set a threshold for 'detecting' the otoliths, set it at `level' of the maximal brightness of the smoothed image
    if not threshold:
        threshold = level * smoothed.max()
    # Detect peaks in smoothed image, in x- and y-direction
    x = numpy.mean(smoothed > threshold, axis=0)
    y = numpy.mean(smoothed > threshold, axis=1)
    peaksx, _ = scipy.signal.find_peaks(x)
    peaksy, _ = scipy.signal.find_peaks(y)
    if verbose:
        plt.subplot(131)
        plt.imshow(img)
        plt.title('Input image')
        plt.subplot(132)
        plt.semilogy(histogram(img), label='Histogram')
        plt.axvline(threshold, label='%s%% of max@%s' % (int(100 * level), int(round(threshold))), color=seaborn.color_palette()[1])
        plt.legend()
        plt.title('Histogram')
        plt.subplot(133)
        plt.imshow(smoothed)
        plt.title('Smoothed image\nwith location of otoliths')
        plt.imshow(numpy.ma.masked_less(img, threshold), cmap='viridis', alpha=0.618)
        for c, p in enumerate(peaksx):
            plt.axvline(p, alpha=0.618)
        for p in peaksy:
            plt.axhline(p, alpha=0.618)
        plt.axvline(numpy.mean(peaksx), color='r')
        plt.axhline(numpy.mean(peaksy), color='r')
        plt.show()
    return([peaksx.tolist(), peaksy.tolist()])

In [ ]:
# Make a column for saving the otolith positions
for d, direction in enumerate(directions):
    Data['Otholith_Positions_' + direction] = ''
    Data['Otholith_Position_Mean_' + direction] = ''

In [ ]:
# Detect otolith positions
for c, row in Data.iterrows():
    print('Finding otolith position for %s/%s' % (row.Fish, row.Scan))
    for d, direction in enumerate(directions):
        Data.at[c, 'Otholith_Positions_' + direction] = otolither(row['MIP_' + direction], verbose=True)

In [ ]:
# Save us the mean position
for d, direction in enumerate(directions):
    Data['Otholith_Position_Mean_' + direction] = [(numpy.mean(op[0]),
                                                    numpy.mean(op[1])) for op in Data['Otholith_Positions_' + direction]]

In [ ]:
for i in Data['Otholith_Positions_Lateral']:
    print(round(numpy.mean(i[0])),
          round(numpy.mean(i[1])))

In [ ]:
for i in Data['Otholith_Position_Mean_Lateral']:
    print(round(i[0]), round(i[1]))

In [ ]:
for whichone in range(len(Data)):
    print(whichone, os.path.join(Data.Fish[whichone], Data.Scan[whichone]))

In [ ]:
direction = 'Lateral'

In [ ]:
for c, row in Data.iterrows():
    for d, direction in enumerate(directions):
        plt.subplot(1, 3, d + 1)
        plt.imshow(row['MIP_' + direction])
        plt.title([round(i) for i in row['Otholith_Position_Mean_' + direction]])
        plt.axhline(row['Otholith_Position_Mean_' + direction][1])
        plt.axvline(row['Otholith_Position_Mean_' + direction][0])
        plt.suptitle('%s/%s' % (row.Fish, row.Scan))
        plt.gca().add_artist(ScaleBar(row.Voxelsize, 'um'))
    plt.show()

In [ ]:
# https://stackoverflow.com/a/50011743/323100
def rescale_linear(array, new_min, new_max):
    """Rescale an arrary linearly."""
    minimum, maximum = numpy.min(array), numpy.max(array)
    m = (new_max - new_min) / (maximum - minimum)
    b = new_min - m * minimum
    return m * array + b

In [ ]:
# Detect otollith position by looking for maximum gray value along fish
for d, direction in enumerate(directions):
    Data['GrayValueAlong_' + direction] = ''
    Data['Otolith_MIP_Position_' + direction] = ''
for whichone, row in Data.iterrows():
    for d, direction in enumerate(directions):
        # Calculate gray value sum along fish.
        Data.at[whichone, 'GrayValueAlong_' + direction] = dask.array.sum(Data['MIP_' + direction][whichone],
                                                                          axis=1)
        # Maximum of this shoud give us the otolith position
        Data.at[whichone,
                'Otolith_MIP_Position_' + direction] = dask.array.argmax(dask.array.sum(Data['MIP_' + direction][whichone],
                                                                                        axis=1))
        # Plot what we found
        plt.subplot(1, 3, d + 1)
        plt.imshow(Data['MIP_' + direction][whichone])
        # Plot the *rescaled* values over the image
        plt.plot(rescale_linear(Data['GrayValueAlong_' + direction][whichone],
                                100,
                                Data['MIP_' + direction][whichone].shape[1] - 100),
                 range(len(Data['GrayValueAlong_' + direction][whichone])),
                 label='Normalized gray value sum along fish',
                 color=seaborn.color_palette()[0])
        plt.axhline(Data['Otolith_MIP_Position_' + direction][whichone],
                    label='Max@%s' % Data['Otolith_MIP_Position_' + direction][whichone].compute(),
                    color=seaborn.color_palette()[1])
        plt.title('%s MIP' % direction)
        plt.suptitle('%s/%s: MIPs of %s/%s' % (whichone, len(Data), Data.Fish[whichone], Data.Scan[whichone]))
        plt.legend(loc='lower center')
    plt.show()

In [ ]:
for c, row in Data.iterrows():
    print(c, len(Data), os.path.join(row.Fish, row.Scan))
    print('\t Otolith from MIP',
          row['Otolith_MIP_Position_Anteroposterior'].compute(),
          row['Otolith_MIP_Position_Lateral'].compute(),
          row['Otolith_MIP_Position_Dorsoventral'].compute())
    print('\t Otolith from otholither function',
          row['Otholith_Position_Mean_Anteroposterior'],
          row['Otholith_Position_Mean_Lateral'],
          row['Otholith_Position_Mean_Dorsoventral'])

In [ ]:
whichone = 10

In [ ]:
for c, direction in enumerate(directions):
    plt.subplot(1, 3, c + 1)
    plt.imshow(Data['MIP_' + direction][whichone])
    # From otholither function
    plt.axhline(Data['Otholith_Position_Mean_' + direction][whichone][1],
                label='otholither mean position 1: %s' % round(Data['Otholith_Position_Mean_' + direction][whichone][1]),
                color=seaborn.color_palette()[0])
    plt.axvline(Data['Otholith_Position_Mean_' + direction][whichone][0],
                label='otholither mean posistion 0: %s' % round(Data['Otholith_Position_Mean_' + direction][whichone][0]),
                color=seaborn.color_palette()[1])
    # From sum along axis
    plt.axhline(Data['Otolith_MIP_Position_' + direction][whichone],
                label='MIP sum: %s' % Data['Otolith_MIP_Position_' + direction][whichone].compute(),
                color=seaborn.color_palette()[3])
    plt.legend(loc='lower center')
    plt.title(direction)
    plt.suptitle('%s/%s' % (Data.Fish[whichone], Data.Scan[whichone]))
    plt.gca().add_artist(ScaleBar(Data.Voxelsize[whichone], 'um'))
plt.show()

In [ ]:
for direction in directions:
    print(direction, Data['MIP_' + direction][whichone].shape)

In [ ]:
for direction in directions:
    print(direction, Data['Otholith_Position_Mean_' + direction][whichone])

In [ ]:
for direction in directions:
    print(direction, round(Data['Otolith_MIP_Position_' + direction][whichone].compute()))

In [ ]:
# CAN WE CALCULATE BOTH DV and THE LT POSITION ON THE AP MIP?

In [ ]:
# Get us positions of otolith in relation to original data
position_ap = numpy.mean((Data['Otholith_Position_Mean_Lateral'][whichone][1],
                          Data['Otholith_Position_Mean_Dorsoventral'][whichone][1],
                          Data['Otolith_MIP_Position_Lateral'][whichone],
                          Data['Otolith_MIP_Position_Dorsoventral'][whichone]),
                         dtype='int')
# laterally, we assume the center of the image for now
# position_lt = numpy.mean((Data['Otholith_Position_Mean_' + direction][whichone][1],
#                           Data['Otolith_MIP_Position_' + direction][whichone]),dtype='int')
position_lt = Data.MIP_Dorsoventral[whichone].shape[1] // 2
position_dv = numpy.mean((Data['Otholith_Position_Mean_Anteroposterior'][whichone][0],
                          Data['Otholith_Position_Mean_Lateral'][whichone][0]),
                         dtype='int')

In [ ]:
print(position_ap)
print(position_lt)
print(position_dv)

In [ ]:
slicethickness = 400
for c, direction in enumerate(directions):
    plt.subplot(1, 3, c + 1)
    plt.imshow(Data['MIP_' + direction][whichone])
    if c == 0:
        plt.fill_between(range(Data['MIP_' + direction][whichone].shape[1]),
                         position_lt - slicethickness // 2,
                         position_lt + slicethickness // 2,
                         alpha=0.5)
        plt.fill_between(range(position_dv - slicethickness // 2, position_dv + slicethickness // 2),
                         1,
                         Data['MIP_' + direction][whichone].shape[0] - 1,
                         alpha=0.5)
    elif c == 1:
        plt.fill_between(range(Data['MIP_' + direction][whichone].shape[1]),
                         position_ap - slicethickness // 2,
                         position_ap + slicethickness // 2,
                         alpha=0.5)
        plt.fill_between(range(position_dv - slicethickness // 2, position_dv + slicethickness // 2),
                         1,
                         Data['MIP_' + direction][whichone].shape[0] - 1,
                         alpha=0.5)
    else:
        plt.fill_between(range(Data['MIP_' + direction][whichone].shape[1]),
                         position_ap - slicethickness // 2,
                         position_ap + slicethickness // 2,
                         alpha=0.5)
        plt.fill_between(range(position_lt - slicethickness // 2, position_lt + slicethickness // 2),
                         1,
                         Data['MIP_' + direction][whichone].shape[0] - 1,
                         alpha=0.5)
    plt.title(direction)
    plt.suptitle('%s/%s' % (Data.Fish[whichone], Data.Scan[whichone]))
    plt.gca().add_artist(ScaleBar(Data.Voxelsize[whichone], 'um'))
plt.show()

In [ ]:
# Grab region calculated above from reconstructions
slicethickness = 888
otolithregion = Reconstructions[whichone][position_ap - slicethickness // 2:position_ap + slicethickness // 2,
                                          position_lt - slicethickness // 2:position_lt + slicethickness // 2,
                                          position_dv - slicethickness // 2:position_dv + slicethickness // 2].compute()
for ax in range(3):
    plt.subplot(1, 3, ax + 1)
    plt.imshow(dask.array.max(otolithregion, axis=ax))
    plt.suptitle('%s/%s: MIP from AP %s:%s, LT %s:%s, DV %s:%s' % (Data.Fish[whichone],
                                                                   Data.Scan[whichone],
                                                                   position_ap - slicethickness // 2, position_ap + slicethickness // 2,
                                                                   position_lt - slicethickness // 2, position_lt + slicethickness // 2,
                                                                   position_dv - slicethickness // 2, position_dv + slicethickness // 2))
plt.show()

In [ ]:
threshold(otolithregion)

In [ ]:
threshold(otolithregion[otolithregion > 65])

In [ ]:
plt.imshow(otolithregion[444])
plt.imshow(otolithregion[444] > 106, cmap='viridis', alpha=0.5)

In [ ]:
# import pickle

In [ ]:
# file = open('largest', 'wb')
# pickle.dump(largest,file)
# file.close()

In [ ]:
# file = open('largest', 'rb')
# largest = pickle.load(file)
# file.close()

In [ ]:
# largest.shape

In [ ]:
# # Make file smaller for testing reasons
# subsample = 4
# largest_smaller = largest[::subsample, ::subsample, ::subsample]
# largest_smaller.shape

In [ ]:
subsample = 4

In [ ]:
vmin = threshold(otolithregion)
print(vmin)

In [ ]:
# Load into K3D
plt_volume = k3d.volume(otolithregion[::subsample, ::subsample, ::subsample].astype(numpy.float16),
                        color_range=[vmin, 2**8])

In [ ]:
# Display the otolith
plot = k3d.plot()
plot += plt_volume
plot.display()